In [1]:
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5103)
Using Theano backend.


In [2]:
data_path = 'data/state/'
sample_path = 'data/state/sample/'
path = data_path
model_path = path + 'models/'
results_path = path + 'results/'
home_dir = os.path.realpath('.')
for p in [path, model_path, sample_path, results_path]:
    if not os.path.exists(p):
        os.makedirs(p)

In [3]:
train_folder = data_path + 'train/'
valid_folder = data_path + 'valid/'
sample_train_folder = sample_path + 'train/'
sample_valid_folder = sample_path + 'valid/'
batch_size = 64

### Load batches

In [ ]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow_from_directory(sample_train_folder, target_size=(224, 224), batch_size=batch_size, shuffle=True)
valid_batches = gen.flow_from_directory(sample_valid_folder, target_size=(224, 224), batch_size=batch_size, shuffle=True)

In [4]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(path)

Found 17082 images belonging to 10 classes.
Found 5342 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


### Load cached data

In [ ]:
trn = get_data(path+'train')
val = get_data(path+'valid')

In [ ]:
save_array(results_path+'val.dat', val)
save_array(results_path+'trn.dat', trn)

In [ ]:
val = load_array(results_path+'val.dat')
trn = load_array(results_path+'trn.dat')

## Prepare Dataset

In [ ]:
# Download
%cd {path}
!kg download -c 'state-farm-distracted-driver-detection'
%cd {home_dir}

In [ ]:
_, dirs, _ = next(os.walk(train_folder))
def copy_dirs(dirs, target):
    for d in dirs:
        if not os.path.exists(target+d):
            os.makedirs(target+d)

copy_dirs(dirs, valid_folder)
copy_dirs(dirs, sample_valid_folder)
copy_dirs(dirs, sample_train_folder)

In [ ]:
# Unzip training labels
import zipfile
def unzip_file(filename, path, outputpath=None):
    outputpath = outputpath or path
    filepath = path + filename
    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
    if os.path.isfile(filepath):
        zip_ref = zipfile.ZipFile(filepath, 'r')
        zip_ref.extractall(outputpath)
        zip_ref.close()
        print('Unzipping file:', filepath)

unzip_file('imgs.zip', path)
unzip_file('sample_submission.csv.zip', path)
unzip_file('driver_imgs_list.csv.zip', path)

### Create Validation set and Sample

In [ ]:
# Get training labels
drivers_list_df = pd.read_csv(path + 'driver_imgs_list.csv')
size = drivers_list_df.size
# print(drivers_list_df)
print('Training set size:', size)


In [ ]:
# print(drivers_list_df.groupby(['subject']).agg(['count']))
driver_ids = drivers_list_df['subject'].unique();
valid_driver_ids = np.random.choice(driver_ids, int(driver_ids.shape[0] * .25))
print('Driver IDs used for validation set:', valid_driver_ids)
valid_df = drivers_list_df.loc[drivers_list_df['subject'].isin(valid_driver_ids)]
train_df = drivers_list_df.loc[~drivers_list_df['subject'].isin(valid_driver_ids)]

for row in valid_df.itertuples():
    filename = row[2] + '/' + row[3]
    os.rename(train_folder+filename, valid_folder+filename)
    
from shutil import copyfile
for row in valid_df.sample(1000).itertuples():
    filename = row[2] + '/' + row[3]
    copyfile(valid_folder+filename, sample_valid_folder+filename)
    
for row in train_df.sample(1500).itertuples():
    filename = row[2] + '/' + row[3]
    copyfile(train_folder+filename, sample_train_folder+filename)

## Basic models

In [ ]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3,) + batches.target_size),
    Flatten(),
    Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(lr=0.001), metrics=['accuracy'], loss='categorical_crossentropy')
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                    validation_data=valid_batches, nb_val_samples=valid_batches.nb_sample)

In [ ]:
np.round(model.predict_generator(batches, batches.nb_sample)[:10])

In [ ]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3,) + batches.target_size),
    Flatten(),
    Dense(10, activation='softmax')
])
model.compile(optimizer=Adam(lr=0.00001), metrics=['accuracy'], loss='categorical_crossentropy')

In [ ]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                    validation_data=valid_batches, nb_val_samples=valid_batches.nb_sample)

### Hidden layer with regularization

In [ ]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3,) + batches.target_size),
    Flatten(),
    Dense(200, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax', W_regularizer=l2(0.01))
])
model.compile(optimizer=Adam(lr=0.00001), metrics=['accuracy'], loss='categorical_crossentropy')

In [ ]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                    validation_data=valid_batches, nb_val_samples=valid_batches.nb_sample)

In [ ]:
model.optimizer.lr = 0.01
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                    validation_data=valid_batches, nb_val_samples=valid_batches.nb_sample)

### Convolutional Model

In [ ]:
def conv2(batches):
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,)+batches.target_size),
        Convolution2D(32, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Convolution2D(64, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
#         Dense(10, activation='softmax', W_regularizer=l2(0.01))
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=Adam(lr=0.00001), metrics=['accuracy'], loss='categorical_crossentropy')
    model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                        validation_data=valid_batches, nb_val_samples=valid_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                        validation_data=valid_batches, nb_val_samples=valid_batches.nb_sample)
    return model

In [ ]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=valid_batches, 
                     nb_val_samples=valid_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=valid_batches, 
                     nb_val_samples=valid_batches.nb_sample)
    return model


### Data augmentation

In [ ]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(sample_train_folder, gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

### VGG

In [4]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3, 224, 224)),
    Convolution2D(32, 3, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Convolution2D(64, 3, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Convolution2D(128, 3, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Flatten(),
    Dense(1000, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

Use VGG model instead

In [5]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [idx for idx, layer in enumerate(model.layers) if type(layer) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [6]:
conv_model = Sequential(conv_layers)

In [7]:
batches = get_batches(path+'train', batch_size=batch_size, shuffle=False)
val_batches = get_batches(path+'valid', batch_size=batch_size, shuffle=False)
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False, class_mode=None)

Found 17082 images belonging to 10 classes.
Found 5342 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [8]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 17082 images belonging to 10 classes.
Found 5342 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [ ]:
print(conv_feat.shape)
print(batches.image_shape)
print(conv_model.output_shape)
(100,) + conv_model.output_shape[1:]

In [32]:
import math
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

def predict_with_progress(model, batches, nb_samples=None):
    num_samples = nb_samples or batches.nb_sample
    batch_size = batches.batch_size

    p_results = np.zeros((num_samples,)+model.output_shape[1:])
    current_index = 0
    # Iterative loop
    for batch in tqdm(batches, total=math.ceil(num_samples/batch_size)):
        if batch is None:
            break
        if type(batch) is tuple:
            batch = batch[0]
        p = model.predict_on_batch(batch)
        p_size = p.shape[0]
#         print('Predictions: {}\n Size: {}'.format(p_true, p_size))
        new_index = current_index + p_size
#         print('Current index: {} New index: {} PResults: {}'.format(current_index, new_index, p_results))
        p_results[current_index:new_index] = p
        current_index = new_index
        if current_index >= num_samples:
            break
    return p_results

In [ ]:
# conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_feat = predict_with_progress(conv_model, batches)

In [27]:
# conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_val_feat = predict_with_progress(conv_model, val_batches)

(64, 3, 224, 224)
(64, 3, 224, 224)
(64, 3, 224, 224)
(64, 3, 224, 224)
(64, 3, 224, 224)
(64, 3, 224, 224)
(64, 3, 224, 224)



KeyboardInterrupt: 

In [35]:
# conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)
conv_test_feat = predict_with_progress(conv_model, test_batches, int(test_batches.nb_sample/2))

KeyboardInterrupt: 

In [18]:
conv_test_feat.shape
print(test_batches.class_mode)
print(val_batches.class_mode)

None
categorical


In [ ]:
save_array(results_path+'conv_val_feat.dat', conv_val_feat)
save_array(results_path+'conv_test_feat.dat', conv_test_feat)
save_array(results_path+'conv_feat.dat', conv_feat)

In [ ]:
conv_val_feat = None
conv_feat = None

In [ ]:
def get_bn_layers(p):
    Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(10, activation='softmax')
    ])

In [ ]:
p = 0.8

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1,
             validation_data=(conv_val_feat, val_labels))

In [ ]:
vn_model.optimzer.lr = 0.01

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=2,
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.sae_weights(path+'models/conv8.h5')

### Pre-computed data augmentation + dropout

In [30]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 17082 images belonging to 10 classes.


In [39]:
da_conv_feat = predict_with_progress(conv_model, da_batches, da_batches.nb_sample*4)

KeyboardInterrupt: 